<a href="https://colab.research.google.com/github/joojinho97/2022_GC_Covid19_prediction/blob/main/3D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install medpy

     |████████████████████████████████| 151 kB 13.9 MB/s 
     |████████████████████████████████| 48.4 MB 1.2 MB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754445 sha256=71d63b0c243788e095834a09bf895a28d7214a26f1472360f04f84b67ca8c833
  Stored in directory: /root/.cache/pip/wheels/b0/57/3a/da1183f22a6afb42e11138daa6a759de233fd977a984333602
Successfully built medpy


In [2]:
import medpy
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import pandas as pd
import numpy as np
import cv2
import SimpleITK as sitk
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense,Flatten,Conv2D,Concatenate,MaxPooling2D,BatchNormalization,Activation,Add,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB7,EfficientNetB6,EfficientNetB5,EfficientNetB3,EfficientNetB1,ResNet101,ResNet50

In [3]:
from medpy.io import load

In [22]:

os.chdir('/content/drive/MyDrive/grand_challenge/destination/data/mha/')
data_list=glob.glob('*.mha')
data_list.sort(key=lambda fname: int(fname.split('.')[0]))
for i in range(len(data_list)):
  data_list[i]='/content/drive/MyDrive/grand_challenge/destination/data/mha/'+data_list[i]

In [23]:
data=pd.read_csv('/content/drive/MyDrive/grand_challenge/destination/metadata/reference.csv')
data=data.sort_values(by='PatientID')


'''
num = np.unique(data_covid, axis=0)
num = num.shape[0]
encoding_covid = np.eye(num)[data_covid]
  
num = np.unique(data_severe, axis=0)
num = num.shape[0]
encoding_severe = np.eye(num)[data_severe]

encoding_covid=np.array(encoding_covid)
encoding_severe=np.array(encoding_severe)
y_multilabel = np.concatenate([encoding_covid, encoding_severe], axis=0)
'''

'\nnum = np.unique(data_covid, axis=0)\nnum = num.shape[0]\nencoding_covid = np.eye(num)[data_covid]\n  \nnum = np.unique(data_severe, axis=0)\nnum = num.shape[0]\nencoding_severe = np.eye(num)[data_severe]\n\nencoding_covid=np.array(encoding_covid)\nencoding_severe=np.array(encoding_severe)\ny_multilabel = np.concatenate([encoding_covid, encoding_severe], axis=0)\n'

In [24]:
tr=list(zip(data["probCOVID"].tolist(), data["probSevere"].tolist()))

In [25]:
train_list=data_list[:1400]
val_list=data_list[1401:]
train_label=tr[:1400]
val_label=tr[1401:]

In [19]:
test_list=data_list[401:]
test_label=tr[401:]

In [53]:
from typing import Iterable
def clip_and_normalize(np_image: np.ndarray,
                       clip_min: int = -703,
                       clip_max: int = -368
                       ) -> np.ndarray:
    np_image=np.transpose(np_image)
    
    np_image[np_image<clip_min]=0
    np_image[np_image>clip_max]=0
    #np_image = np.clip(np_image, clip_min, clip_max)
    np_image = np_image/255.0
    np_image=np.abs(np_image)
    #np_image = (np_image - clip_min) / (clip_max - clip_min)
    return np_image


def resample(itk_image: sitk.Image,
             new_spacing: Iterable[float],
             outside_val: float = 0
             ) -> sitk.Image:
             
    
    #밑에 getsize getspacing 어캐 해결하지
    shape = itk_image.GetSize()
    spacing = itk_image.GetSpacing()
    output_shape = tuple(int(round(s * os / ns)) for s, os, ns in zip(shape, spacing, new_spacing))
    return sitk.Resample(
        itk_image,
        output_shape,
        sitk.Transform(),
        sitk.sitkLinear,
        itk_image.GetOrigin(),
        new_spacing,
        itk_image.GetDirection(),
        outside_val,
        sitk.sitkFloat32,
    )


def center_crop(np_image: np.ndarray,
                new_shape: Iterable[int],
                outside_val: float = 0
                ):
  
    
    output_image = np.full(new_shape, outside_val, np_image.dtype)

    slices = tuple()
    offsets = tuple()
    
    for it, sh in enumerate(new_shape):
        size = sh // 2
        if it == 0:
            
            center = np_image.shape[it] - size
            
        else:
            
            center = (np_image.shape[it] // 2)
        start = center - size
        stop = center + size + (sh % 2)

        # computing what area of the original image will be in the cropped output
        slce = slice(max(0, start), min(np_image.shape[it], stop))
        slices += (slce,)

        # computing offset to pad if the crop is partly outside of the scan
        offset = slice(-min(0, start), 2 * size - max(0, (start + 2 * size) - np_image.shape[it]))
        offsets += (offset,)

    output_image[offsets] = np_image[slices]

    return output_image


def preprocess(input_image: sitk.Image,
               new_spacing: Iterable[float] = (1.6, 1.6, 1.6),
               new_shape: Iterable[int] = (240, 240, 240),
               ) -> np.ndarray:

    input_image = resample(input_image, new_spacing=new_spacing)
    input_image = sitk.GetArrayFromImage(input_image)
    input_image = center_crop(input_image, new_shape=new_shape)
    input_image = clip_and_normalize(input_image)
    return input_image

def mha2jpg(mhapath):
        
    mhapath=str(mhapath.numpy())[2:-1]

    reader = sitk.ImageFileReader()
    reader.SetFileName(mhapath)   # Give it the mha file as a string
    reader.LoadPrivateTagsOn()     # Make sure it can get all the info
    reader.ReadImageInformation()
    
    try:
      gender=reader.GetMetaData('PatientSex')
      if gender=='M':
        genders=float(0)
      elif gender=='F':
        genders=float(1)
    except:
      genders=float(0)
    
    try:
      age=reader.GetMetaData('PatientAge')
      age=float(age)
      
      age=int(age)
      if age<40:
        age=0
      elif age<50:
        age=1
      elif age<60:
        age=2
      elif age<70:
        age=3
      elif age<80:
        age=4
      else:
        age=5
    except:
      if age=='035Y':
        age=0
      elif age=='045Y':
        age=1
      elif age=='055Y':
        age=2
      elif age=='065Y':
        age=3
      elif age=='075Y':
        age=4
      elif age=='085Y':
        age=5

    
    input_image=sitk.ReadImage(mhapath)
    
    input_image = resample(input_image, new_spacing=(1.6, 1.6, 1.6))
    input_image=sitk.GetArrayFromImage(input_image)
    
    input_image = center_crop(input_image, new_shape=(3,512,512))
     
    input_image = clip_and_normalize(input_image)
    
    age=np.expand_dims(age,axis=-1)
    try:
      genders=np.expand_dims(genders,axis=-1)
    except:
      genders=0
      genders=np.expand_dims(genders,axis=-1)

    return input_image,age,genders




In [54]:
def data_(dataset):
  
  dataset=dataset.map(data_preprocess)  
  return  dataset

def data_preprocess(x, labels):
  
  
  img,age,gender=tf.py_function(mha2jpg,inp=[x],Tout=(tf.float32,tf.float32,tf.float32))
  return {'input_1' :img, 'age':age,'gender':gender }, {'probCOVID':labels[0], 'probSevere' :labels[1]}


In [55]:
def make_dataset(train_list, tr):
  
  test_flist=tf.data.Dataset.from_tensor_slices((train_list, tr))
  test_dataset=data_(test_flist)
  test_dataset=test_dataset.batch(1)
  test_dataset=test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
  
  return test_dataset


In [56]:
train_dataset=make_dataset(train_list, train_label)
val_dataset=make_dataset(val_list,val_label)

In [57]:
input_=Input(shape=(512,512,3),name='input_1')
input_shape=(512,512,3)
input_age=Input(shape=(1,),name='age')


In [58]:
ef=ResNet50(weights=None,include_top=False, input_shape=input_shape, pooling="max")(input_)



predictions_covid = tf.keras.layers.Dense(1, activation='sigmoid',name='probCOVID')(ef)

severe_dense=Concatenate()([ef,input_age,predictions_covid])
predictions_severe = tf.keras.layers.Dense(1, activation='sigmoid',name='probSevere')(severe_dense)


model = tf.keras.Model(inputs=[input_,input_age], outputs=[predictions_covid,predictions_severe])

In [59]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss={'probCOVID': 'binary_crossentropy', 'probSevere':'binary_crossentropy'},metrics=[tf.keras.metrics.AUC(multi_label=True, curve='ROC', name='auc_pr')])

In [60]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./new_{epoch:02d}.h5', verbose=True, save_weights_only=True, mode='auto'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.66, patience=2, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0),
]

In [ ]:
model.fit(train_dataset, epochs=10,validation_data=val_dataset,callbacks=callbacks)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['gender'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


1400/1400 [==============================] - ETA: 0s - loss: 4.1497 - probCOVID_loss: 2.5184 - probSevere_loss: 1.6313 - probCOVID_auc_pr: 0.5264 - probSevere_auc_pr: 0.5169
Epoch 00001: saving model to ./new_01.h5
1400/1400 [==============================] - 6676s 5s/step - loss: 4.1497 - probCOVID_loss: 2.5184 - probSevere_loss: 1.6313 - probCOVID_auc_pr: 0.5264 - probSevere_auc_pr: 0.5169 - val_loss: 1.5458 - val_probCOVID_loss: 0.9722 - val_probSevere_loss: 0.5735 - val_probCOVID_auc_pr: 0.4996 - val_probSevere_auc_pr: 0.6193 - lr: 0.0050
Epoch 2/10
 291/1400 [=====>........................] - ETA: 55:58 - loss: 1.2796 - probCOVID_loss: 0.8093 - probSevere_loss: 0.4703 - probCOVID_auc_pr: 0.4716 - probSevere_auc_pr: 0.5459

In [20]:
test_dataset=make_dataset(test_list,test_label)

In [21]:
pr=model.evaluate(test_dataset)

  46/1599 [..............................] - ETA: 1:30:19 - loss: 1.2762 - probCOVID_loss: 0.7183 - probSevere_loss: 0.5579 - probCOVID_auc_pr: 0.5063 - probSevere_auc_pr: 0.4707

KeyboardInterrupt: ignored

In [ ]:
def mha2jpg(mhapath,windowsCenter,windowsSize):
    
    
    
    
    mhapath=str(mhapath)
    
    img_data=sitk.GetArrayFromImage(sitk.ReadImage(mhapath))
    
    channel = img_data.shape[0]

    windowsCenter=np.array(windowsCenter)
    windowsSize=np.array(windowsSize)
    low = windowsCenter-windowsSize/2
    high = windowsCenter+windowsSize/2
    
    if channel != 1177:
      img_data=np.array(img_data)
      img_data=cv2.resize(img_data,(512,512,1177))
     
    

    return img_data




In [ ]:
co=mha2jpg('/content/drive/MyDrive/grand_challenge/destination/data/mha/504.mha',100,200)

TypeError: ignored

In [ ]:
co.shape